<a href="https://colab.research.google.com/github/leandrocodes/Paralel-Programming-UFMS-CPPP/blob/master/T3_PP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation Setup


In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin